# Predicting Meteoroid Conditions

## Preliminar

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy.optimize import minimize
from scipy.interpolate import interp1d as interpolate
from scipy.integrate import quad as integrate
import statistics as stats
import spiceypy as spy
from copy import deepcopy
from mpl_toolkits.basemap import Basemap as Map,shiftgrid as Grid
from tqdm import tqdm
from scipy.stats import linregress
from glob import glob
%matplotlib nbagg

#Routines
def ipercs(xs,ps,qpercs):
    """
    Compute percentiles from a numerical normalized distribution ps
    """
    Ps=np.array([ps[:i].sum() for i in range(len(ps))])
    fi=interpolate(Ps,xs)
    return fi(np.array(qpercs)/100.0)

#Routines to convert from local to body systems and viceversa
def loc2rec(vp,distance,Az,h):
    """
    Convert distance, azimuth and elevation to position w.r.t. Earth
    vp (dictionary): Vantage point.  It should have: lon, lat, alt
    distance: km
    Az: deg
    h: deg
    """
    rlocal=spy.latrec(distance,Az*DEG,h*DEG)
    rpos=spy.mxv(vp["l2b"],rlocal)+vp["geopos"]
    return rpos


def rec2loc(vp,rpos):
    """
    Convert position w.r.t. Earth to distance, Azimuth and elevation
    distance: km
    Az: deg
    h: deg
    """
    rrel=rpos-vp["geopos"]
    rlocal=spy.mxv(vp["b2l"],rrel)
    distance,Az,h=spy.reclat(rlocal)
    Az=2*np.pi+Az if Az<0 else Az
    return distance,Az*RAD,h*RAD

def updateVantagePoint(vp):
    """
    Compute related geometrical properties of a vantage point
    """
    vp["geopos"]=spy.georec(vp["lon"]*DEG,vp["lat"]*DEG,vp["alt"],RE,F)
    normal=spy.surfnm(RE,RE,RP,vp["geopos"])
    uy=spy.ucrss(np.array([0,0,1]),normal)
    ux=spy.ucrss(normal,uy)
    vp["l2b"]=np.array(np.vstack((ux,uy,normal)).transpose().tolist())
    vp["b2l"]=np.linalg.inv(vp["l2b"])

### SPICE specifics

In [2]:
BDIR="./"
spy.furnsh(BDIR+"kernels/de430.bsp")
spy.furnsh(BDIR+"kernels/naif0012.tls")
spy.furnsh(BDIR+"kernels/pck00010.tpc")
spy.furnsh(BDIR+"kernels/earth_070425_370426_predict.bpc")
spy.furnsh(BDIR+"kernels/earth_assoc_itrf93.tf")
#Astronomical constants
n,rs=spy.bodvrd("EARTH","RADII",3)
RE=rs[0];RP=rs[2]
F=(RE-RP)/RE

### Macro and Constants

In [3]:
#Directories
FIGDIR=BDIR+"figures/"

#Macros
norm=np.linalg.norm
DEG=np.pi/180
RAD=1/DEG

#Fitting procedure
lengthTestTrajectory=300 #km
npointsTestTrajectory=100

## Impact data

In [4]:
#Read vantage positions position
havana=pd.read_csv("data/havana-observations-final.csv",decimal=",")
florida=pd.read_csv("data/florida-observations-final.csv",decimal=",")
pinar=pd.read_csv("data/pinardelrio-observations-final.csv",decimal=",")

#Build vantage points dictionary
vantagePoints=dict(
    impact=dict(
        location="Impact",
        obstraj=None,
        lon=0,lat=0,alt=0,
        Azref=0.0,
    ),
    havana=dict(
        location="Havana harbor",
        obstraj=havana,
        lon=-82.344343,lat=23.13799,alt=0/1000.,
        Azref=0.0,
    ),
    florida=dict(
        location="GullWing Beach Resort (Florida)",
        obstraj=florida,
        lon=-81.903596,lat=26.418871,alt=1/1000.,
        Azref=201.6,
    ),
    pinar=dict(
        location="Pinar del Rio (Cuba)",
        obstraj=pinar,
        lon=-83.692091,lat=22.414536,alt=48/1000.,
        Azref=0.0,
    ),
)

#Geometrical properties of the vantage point
for key,vp in vantagePoints.items():updateVantagePoint(vp)
nvp=len(vantagePoints.keys())-1

## Chi-square computation

In [ ]:
def chiSquare(param,vps,avoid=[],verbose=False,npoints=npointsTestTrajectory):
    
    #Input parameters
    lonProj=param[0] # deg
    latProj=param[1] # deg
    hrad=param[2] #deg 
    Azrad=param[3] #deg
    
    #Impact position
    vantagePoints["impact"]["lon"]=lonProj
    vantagePoints["impact"]["lat"]=latProj
    vantagePoints["impact"]["alt"]=0.0
    updateVantagePoint(vantagePoints["impact"])
    
    #Reference Azimuth in Florida
    vantagePoints["florida"]["Azref"]=param[4]
    
    #Direction of radiant
    r=loc2rec(vps["impact"],1,Azrad,hrad)-vps["impact"]["geopos"]
    uimp,magn=spy.unorm(r)
    
    #Trajectory with this conditions
    trajectory=np.zeros((npoints,3))
    for i,t in enumerate(np.linspace(0,lengthTestTrajectory,npoints)):
        trajectory[i]=uimp*t+vps["impact"]["geopos"]
        
    #Az,h points of trayectory in vantage point vp
    chi2=0
    for key,vp in vps.items():
        if key=="impact" or key in avoid:continue

        #Compute theoretical trajectory
        vp["theotraj"]=np.zeros((npoints,2))
        for i in range(len(trajectory)):
            rpos=trajectory[i]
            d,Az,h=rec2loc(vp,rpos)
            vp["theotraj"][i]=np.array([Az,h])
        
        #Compute chisquare for this vantage point
        otraj=vp["obstraj"].values
        dAzmat=np.array([((otraj[i,2]+vp["Azref"])-vp["theotraj"][:,0])**2/otraj[i,3]**2 for i in range(len(otraj))])
        dhmat=np.array([(otraj[i,0]-vp["theotraj"][:,1])**2/otraj[i,1]**2 for i in range(len(otraj))])
        dposmat=np.sqrt(dAzmat+dhmat)
        chi=dposmat.min(axis=1).sum()
        chi2+=(dposmat.min(axis=1).sum()) #/len(otraj)
            
    return chi2

### Minimization

In [ ]:
#Minimization
lonProj=-83.7751101
latProj=22.80728391
hrad=33.0 #deg 
Azrad=180.0 #deg
Azref=192.0 #deg
param=[lonProj,latProj,hrad,Azrad,Azref]
solution=minimize(chiSquare,param,args=(vantagePoints,[],False),options=dict(maxiter=100))

#Plot solution
fig,axs=plt.subplots(nvp,figsize=(4,6))
i=-1

for i,ax in enumerate(axs):
    ax.set_xlabel("Azimuth (deg)")
    ax.set_ylabel("Elevation (deg)")

axs[0].set_title(f"Trajectory solution: Az = {solution.x[3]:.2f}, h = {solution.x[2]:.2f}",fontsize=10)

i=-1
for key,vp in vantagePoints.items():
    if key=="impact":continue
    i+=1
    
    #Plot observed points
    axs[i].plot(vp["theotraj"][:,0],vp["theotraj"][:,1],'k-')
    axs[i].errorbar(vp["obstraj"]["Az"]+vp["Azref"],vp["obstraj"]["h"],
                    xerr=vp["obstraj"]["Azerr"],yerr=vp["obstraj"]["herr"],marker='s',ms=3,ls='')
    #axs[i].text(0.9,0.9,vp["location"],ha='right',va='top',transform=axs[i].transAxes)

    #Ranges
    Azmin=(vp["obstraj"]["Az"]+vp["Azref"]).min();Azmax=(vp["obstraj"]["Az"]+vp["Azref"]).max()
    hmin=0;hmax=vp["obstraj"]["h"].max()
    dAz=(Azmax-Azmin)/5.0
    dh=(hmax-hmin)/5.0
    
    axs[i].set_xlim((Azmin-dAz,Azmax+dAz))
    axs[i].set_ylim((hmin-dh,hmax+dh))
    axs[i].grid()
    
fig.tight_layout()
solution

## Improved plot

In [ ]:
param=solution.x
chiSquare(param,vantagePoints,npoints=100)

In [ ]:
#Trajectory with this conditions
npoints=200

#Direction of radiant
r=loc2rec(vantagePoints["impact"],1,param[3],param[2])-vantagePoints["impact"]["geopos"]
uimp,magn=spy.unorm(r)
trajectory=np.zeros((npoints,3))
for i,t in enumerate(np.linspace(0,lengthTestTrajectory,npoints)):
    trajectory[i]=uimp*t+vantagePoints["impact"]["geopos"]

for key,vp in vantagePoints.items():
    if key=="impact":continue
    vp["theotraj"]=np.zeros((npoints,2))
    for i in range(len(trajectory)):
        rpos=trajectory[i]
        d,Az,h=rec2loc(vp,rpos)
        vp["theotraj"][i]=np.array([Az,h])

In [ ]:
#Plot solution
fig,axs=plt.subplots(nvp,figsize=(5,10))
i=-1

for i,ax in enumerate(axs):
    ax.set_xlabel("Azimuth (deg)")
    ax.set_ylabel("Elevation (deg)")

#axs[0].set_title(f"Trajectory solution: Az = {solution.x[3]:.2f}, h = {solution.x[2]:.2f}",fontsize=10)

i=-1
for key,vp in vantagePoints.items():
    if key=="impact":continue
    i+=1
    
    #Plot observed points
    axs[i].plot(vp["theotraj"][:,0],vp["theotraj"][:,1],'k-')
    axs[i].errorbar(vp["obstraj"]["Az"]+vp["Azref"],vp["obstraj"]["h"],
                    xerr=vp["obstraj"]["Azerr"],yerr=vp["obstraj"]["herr"],marker='s',ms=3,ls='',color='r')
    axs[i].text(0.05,0.1,vp["location"],ha='left',va='bottom',transform=axs[i].transAxes,fontsize=14)

    #Ranges
    Azmin=(vp["obstraj"]["Az"]+vp["Azref"]).min();Azmax=(vp["obstraj"]["Az"]+vp["Azref"]).max()
    hmin=0;hmax=vp["obstraj"]["h"].max()
    dAz=(Azmax-Azmin)/10.0
    dh=(hmax-hmin)/10.0
    
    axs[i].set_xlim((Azmin-dAz,Azmax+dAz))
    axs[i].set_ylim((hmin-dh,hmax+dh))
    axs[i].grid()
    
fig.savefig(FIGDIR+"trajectory-fit.png")
fig.tight_layout()

### Compute the trajectory and the foot print

In [ ]:
vp=vantagePoints["havana"]

In [ ]:
#Range of interpolation
Azmin=(vp["obstraj"]["Az"].values+vp["Azref"]).min()
Azmax=(vp["obstraj"]["Az"].values+vp["Azref"]).max()

#Traverse the theoretical trajectory
dAzmin_min=1e100
dAzmax_min=1e100
imin=0;imax=0
for i in range(len(trajectory)):
    rpos=trajectory[i]
    d,Az,h=rec2loc(vp,rpos)
    dAzmin=np.abs(Az-(Azmin))
    dAzmax=np.abs(Az-(Azmax))
    if dAzmin<=dAzmin_min:
        imin=i
        dAzmin_min=dAzmin
    if dAzmax<=dAzmax_min:
        imax=i
        dAzmax_min=dAzmax
trajectory_sel=trajectory[imax:imin]
len(trajectory_sel)

In [ ]:
trajectory_geo=np.zeros_like(trajectory_sel)
for i,point in enumerate(trajectory_sel):
    lon,lat,alt=spy.recgeo(point,RE,F)
    trajectory_geo[i]=[lon*RAD,lat*RAD,alt]

In [ ]:
f=open(BDIR+"data/cuba-trajectory-Zuluaga_et_al_2019.kml","w")
f.write("""<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/ext/2.2" xmlns:kml="http://www.opengis.net/kml/2.2" xmlns:atom="http://www.w3.org/2005/Atom">
<Document>
	<name>Footprint according to @CMRumpf</name>
	<Style id="sh_ylw-pushpin">
		<IconStyle>
			<scale>1.3</scale>
			<Icon>
				<href>http://maps.google.com/mapfiles/kml/pushpin/ylw-pushpin.png</href>
			</Icon>
			<hotSpot x="20" y="2" xunits="pixels" yunits="pixels"/>
		</IconStyle>
		<BalloonStyle>
		</BalloonStyle>
		<LineStyle>
			<color>ff1bffd9</color>
			<width>1.5</width>
		</LineStyle>
	</Style>
	<StyleMap id="msn_ylw-pushpin">
		<Pair>
			<key>normal</key>
			<styleUrl>#sn_ylw-pushpin</styleUrl>
		</Pair>
		<Pair>
			<key>highlight</key>
			<styleUrl>#sh_ylw-pushpin</styleUrl>
		</Pair>
	</StyleMap>
	<Style id="sn_ylw-pushpin">
		<IconStyle>
			<scale>1.1</scale>
			<Icon>
				<href>http://maps.google.com/mapfiles/kml/pushpin/ylw-pushpin.png</href>
			</Icon>
			<hotSpot x="20" y="2" xunits="pixels" yunits="pixels"/>
		</IconStyle>
		<BalloonStyle>
		</BalloonStyle>
		<LineStyle>
			<color>ff1bffd9</color>
			<width>1.5</width>
		</LineStyle>
	</Style>
	<Placemark>
		<name>Footprint Zuluaga et al. (2019)</name>
		<styleUrl>#msn_ylw-pushpin</styleUrl>
		<LineString>
			<tessellate>1</tessellate>
			<altitudeMode>relativeToGround</altitudeMode>
			<coordinates>
""")

for point in trajectory_geo:
    f.write(f"				{point[0]},{point[1]},{point[2]*1000}\n")
        
f.write("""
			</coordinates>
		</LineString>
	</Placemark>
</Document>
</kml>
""")
f.close()

In [ ]:
f=open(BDIR+"data/cuba-polygon-Zuluaga_et_al_2019.kml","w")
f.write("""<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/ext/2.2" xmlns:kml="http://www.opengis.net/kml/2.2" xmlns:atom="http://www.w3.org/2005/Atom">
<Document>
	<name>Trajectory according to Zuluaga et al. (2019)</name>
	<Style id="visible_area">
		<LineStyle>
			<color>ff1bffd9</color>
			<width>1.5</width>
		</LineStyle>
		<PolyStyle>
			<color>55000000</color>
		</PolyStyle>
	</Style>
	<Placemark>
		<name>Footprint Zuluaga et al. (2019)</name>
		<styleUrl>#msn_ylw-pushpin</styleUrl>
		<Polygon>
			<altitudeMode>relativeToGround</altitudeMode>
			<outerBoundaryIs>
				<LinearRing>
					<coordinates>
""")

f.write(f"				{trajectory_geo[0,0]},{trajectory_geo[0,1]},0\n")
for point in trajectory_geo:
    f.write(f"				{point[0]},{point[1]},{point[2]*1000}\n")
f.write(f"				{trajectory_geo[-1,0]},{trajectory_geo[-1,1]},0\n")
f.write(f"				{trajectory_geo[0,0]},{trajectory_geo[0,1]},0\n")

f.write("""
				</coordinates>
				</LinearRing>
			</outerBoundaryIs>
		</Polygon>
	</Placemark>
</Document>
</kml>
""")
f.close()

## Velocity

In [ ]:
vp=vantagePoints["havana"]
vp["obstraj"]

In [ ]:
#Range of interpolation
Azmin=(vp["obstraj"]["Az"].values+vp["Azref"]).min()
Azmax=(vp["obstraj"]["Az"].values+vp["Azref"]).max()
tmin=vp["obstraj"]["t"].min()
tmax=vp["obstraj"]["t"].max()

i_ini=0;i_end=10
Azmin=(vp["obstraj"]["Az"].values+vp["Azref"])[i_ini]
Azmax=(vp["obstraj"]["Az"].values+vp["Azref"])[i_end]
tmin=vp["obstraj"]["t"][i_ini]
tmax=vp["obstraj"]["t"][i_end]

#Traverse the theoretical trajectory
dAzmin_min=1e100
dAzmax_min=1e100
imin=0;imax=0
for i in range(len(trajectory)):
    rpos=trajectory[i]
    d,Az,h=rec2loc(vp,rpos)
    dAzmin=np.abs(Az-(Azmin))
    dAzmax=np.abs(Az-(Azmax))
    if dAzmin<=dAzmin_min:
        imin=i
        dAzmin_min=dAzmin
    if dAzmax<=dAzmax_min:
        imax=i
        dAzmax_min=dAzmax
trajectory_sel=trajectory[imax:imin]

### Interpolate points in trajectory

In [ ]:
vp=vantagePoints["havana"]

#Build fulltrajectory
fulltrajectory=np.concatenate((trajectory,vp["theotraj"]),axis=1)

#Interpolate positions
xfunc=interpolate(fulltrajectory[:,3],fulltrajectory[:,0])
yfunc=interpolate(fulltrajectory[:,3],fulltrajectory[:,1])
zfunc=interpolate(fulltrajectory[:,3],fulltrajectory[:,2])

#Calculate distance cover between observed points
spoints=np.zeros((len(vp["obstraj"]),5))
for i,point in enumerate(vp["obstraj"].values):
    Az=point[2]
    t=point[4]
    intpoint=[float(xfunc(Az)),float(yfunc(Az)),float(zfunc(Az))]
    if i==0:
        distance=0
    else:
        distance=norm(intpoint-spoints[0,:3])
    spoints[i]=np.array(intpoint+[distance,t])
    
#Ranges
tmin=spoints[:,4].min();tmax=spoints[:,4].max()

#Perform linear regression of times and distances
regression=linregress(spoints[:,4],spoints[:,3])
d0=regression.intercept;vimp=regression.slope

In [ ]:
fig=plt.figure()
ax=fig.gca()

ax.plot(spoints[:,4],spoints[:,3],'ko')
ts=np.linspace(tmin,tmax,100)
ax.plot(ts,d0+vimp*ts,'b-')

ax.set_xlabel("t (s)",fontsize=14)
ax.set_ylabel("d (km)",fontsize=14)

ax.text(0.1,0.9,r"$\langle v_{\rm imp}\rangle=%.2f$ km/s"%vimp,transform=ax.transAxes,fontsize=16)

ymin,ymax=ax.get_ylim()
ax.set_xlim((0,tmax))
ax.set_ylim((0,ymax))

fig.tight_layout()
fig.savefig(FIGDIR+"impact-speed.png")

### LaTeX tables

In [ ]:
for key,vp in vantagePoints.items():
    if key=="impact":continue
    print("\\hline\\multicolumn{4}{c}{%s}\\\\\\hline"%vp['location']) 
    for i,point in enumerate(vp["obstraj"].values):
        if key=="pinar":t="-"
        else:t=f"{point[4]:.2f}"
        if key=="florida":Az=r"$A_{\rm ref}$"+f"+{point[2]:.1f}"
        else:Az=f"{point[2]:.2f}"
            
        #print(f"{i+1} & %.1f $\pm$ {point[3]:.1g} & {point[0]:.1f} $\pm$ {point[1]:.1g} & {t} \\\\"%(point[2]+vp["Azref"]))
        print(f"{i+1} & {Az} $\pm$ {point[3]:.1g} & {point[0]:.1f} $\pm$ {point[1]:.1g} & {t} \\\\"%(point[2]+vp["Azref"]))

        

### Generate initial conditions for gravitational ray tracing

In [ ]:
#Solution
lon=-83.78217648;dlonp=+0.05;dlonm=-0.05
lat=22.80238773;dlatp=+0.05;dlatm=-0.05
Azrad=179;dAzradp=+1.0;dAzradm=-5.0
hrad=33;dhradp=+0.5;dhradm=-1.5
vimp=18.0;dvimpp=+0.3;dvimpm=-1.2

In [ ]:
#Generate GRT files
N=10

#Velocities
f=open("data/velocities.dat.cuba_error","w")
vmax=vimp+dvimpp
vmin=vimp+dvimpm
for i in range(N):
    v=vmin+(vmax-vmin)*np.random.rand()
    f.write(f"{v}\t-1\n")
f.close()

#Directions
f=open("data/directions.dat.cuba_error","w")
Azradmax=Azrad+dAzradp
Azradmin=Azrad+dAzradm
hradmax=hrad+dhradp
hradmin=hrad+dhradm
for i in range(N):
    Az=Azradmin+(Azradmax-Azradmin)*np.random.rand()
    h=hradmin+(hradmax-hradmin)*np.random.rand()
    f.write(f"-1\t-1\t{Az}\t{h}\n")
f.close()

#Locations
f=open("data/locations.dat.cuba_error","w")
lonmax=lon+dlonp
lonmin=lon+dlonm
latmax=lat+dlatp
latmin=lat+dlatm
for i in range(N):
    lon=lonmin+(lonmax-lonmin)*np.random.rand()
    lat=latmin+(latmax-latmin)*np.random.rand()
    f.write(f"-1\t-1\t{lon}\t{lat}\n")
f.close()

In [ ]:
#After running the GRT read all rays
direrror="data/grt-Cuba_error-20190201181700-BE1E80/"
for i,physfile in enumerate(glob(f"{direrror}/*.phys")):
    data=np.loadtxt(physfile)
    if i==0:
        alldata=data
    else:
        alldata=np.concatenate((alldata,data),axis=0)

#Compute semimajor axis, period and Tisserand parameter
aes=alldata[:,9]/(1-alldata[:,10])
Ps=aes**1.5
Tps=1/aes+2*np.cos(alldata[:,11]*DEG)*np.sqrt(aes*(1-alldata[:,10]**2))
alldata=np.column_stack((alldata,aes,Ps,Tps))
#alldata=np.append(alldata,Ps)
#alldata=np.append(alldata,Tps)
        
#Perform analysis
percentiles=np.percentile(alldata,[15,50,85],axis=0)

In [ ]:
props=[
    dict(
        name="a",
        symbol="a",
        units="AU",
        ind=16,
    ),
    dict(
        name="q",
        symbol="q",
        units="AU",
        ind=9,
    ),
    dict(
        name="e",
        symbol="e",
        units="AU",
        ind=10,
    ),
    dict(
        name="i",
        symbol="i",
        units="AU",
        ind=11,
    ),
    dict(
        name="W",
        symbol="\Omega",
        units="AU",
        ind=12,
    ),
    dict(
        name="o",
        symbol="\omega",
        units="AU",
        ind=13,
    ),
    dict(
        name="P",
        symbol="P",
        units="yr",
        ind=17,
    ),
    dict(
        name="Tp",
        symbol="T_p",
        units="adim.",
        ind=18,
    ),
]
for prop in props:
    name=prop["name"]
    pmin,pmed,pmax=percentiles[:,prop["ind"]]
    dpp=pmax-pmed;dpm=pmed-pmin
    print(f"{name} = {pmed:.2f}, d{name} = {(dpp+dpm)/2:.2g},")

In [ ]:
for prop in props:
    name=prop["name"]
    pmin,pmed,pmax=percentiles[:,prop["ind"]]
    dpp=pmax-pmed;dpm=pmed-pmin
    print(f"${prop['symbol']}$ ({prop['units']}) & ${pmed:.2f}^"+"{"+f"+{dpp:.2g}"+"}_{"+f"-{dpm:.2g}"+"}$\\\\")